Next challenge is Marketing Analytics which we should build a model to predict whether a user open emails sent by Shopee Team. This challenge also ends the league held by Shopee Code League. So, i would not share the dataset to prevent any copyright issues with the competition league. Firstly, I am mounting my Google Drive through Google Colab.

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd # import pandas to handle dataframe
import numpy as np # import numpy to perform array operation

Let's read our train data and save it to 'train' dataframe. Later, we also need to read the user dataframe into 'user' dataframe. Then, we need to perform some preprocessing methods. 

In [ ]:
train=pd.read_csv("/content/drive/My Drive/Shopee/Marketing Analytics/train.csv")
train.head() # to see the first 5 data of this dataframe

,country_code,grass_date,user_id,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,open_flag,row_id
0,4,2019-07-16 00:00:00+08:00,43,44,19,6,18,0,2,4,12,43,99,0,5,10,0,0
1,4,2019-07-16 00:00:00+08:00,102,44,9,4,8,2,9,17,18,48,90,1,1,4,1,1
2,6,2019-07-16 00:00:00+08:00,177,49,14,5,5,0,4,12,24,69,119,5,19,27,0,2
3,1,2019-07-16 00:00:00+08:00,184,49,49,9,53,0,0,1,9,23,69,1,3,6,0,3
4,6,2019-07-16 00:00:00+08:00,221,49,227,6,221,0,0,0,2,5,5,0,0,0,0,4


In [ ]:
train.shape # to see the dimension the 'train' dataframe

(73539, 18)

In [ ]:
train['user_id'].unique()

array([    43,    102,    177, ..., 127696, 127807, 127880])

In [ ]:
user=pd.read_csv("/content/drive/My Drive/Shopee/Marketing Analytics/users.csv")
user.head() # to see the first 5 data from 'user'

,user_id,attr_1,attr_2,attr_3,age,domain
0,0,NaN,1.0,0.0,NaN,@gmail.com
1,1,1.0,1.0,2.0,50.0,@gmail.com
2,2,NaN,1.0,0.0,NaN,other
3,3,NaN,1.0,0.0,NaN,@gmail.com
4,4,1.0,1.0,2.0,33.0,@gmail.com


Before we process the main dataset, let's process user dataset which will affect the result of the main dataset. As you can see than in the user dataset, the columns are dominated will unknown columns, containing a lot of NaN, age, and domain. Let's impute the missing value.

In [ ]:
user.isna().sum() # calculate the number of missing values in the dataset

user_id        0
attr_1     48899
attr_2       447
attr_3         0
age        48899
domain         0
dtype: int64

In [ ]:
user.describe() # take a look in the statistical description of the data set. You can see that the minimum age is -17 which is impossible.

,user_id,attr_1,attr_2,attr_3,age
count,127886.000000,78987.000000,127439.000000,127886.000000,78987.000000
mean,63961.168760,0.999823,0.998007,1.079782,31.370124
std,36928.842572,0.013312,0.044600,1.035413,11.591752
min,0.000000,0.000000,0.000000,0.000000,-17.000000
25%,31981.250000,1.000000,1.000000,0.000000,24.000000
50%,63958.500000,1.000000,1.000000,1.000000,29.000000
75%,95941.750000,1.000000,1.000000,2.000000,36.000000
max,127925.000000,1.000000,1.000000,4.000000,118.000000


In [ ]:
user['attr_1'].value_counts() # count the sum of each value in the attr_1 column

1.0    78973
0.0       14
Name: attr_1, dtype: int64

In [ ]:
user['attr_2'].value_counts() # count the sum of each value in the attr_2 column

1.0    127185
0.0       254
Name: attr_2, dtype: int64

From both result above, we can see that they are super dominated with value of 1. So, we fill the missing value of both columns with 1.0

In [ ]:
user['attr_1']=user['attr_1'].fillna(1.0)

In [ ]:
user['attr_2']=user['attr_2'].fillna(1.0)

In [ ]:
user.isna().sum()

user_id        0
attr_1         0
attr_2         0
attr_3         0
age        48899
domain         0
dtype: int64

In [ ]:
user[user['age']<0]=user[user['age']<0]*-1 # for the negative value, we convert it to positive value by multiplying it with -1.

In [ ]:
user.head() # now let's see the latest dataset after applying some processes

,user_id,attr_1,attr_2,attr_3,age,domain
0,0,1.0,1.0,0.0,NaN,@gmail.com
1,1,1.0,1.0,2.0,50.0,@gmail.com
2,2,1.0,1.0,0.0,NaN,other
3,3,1.0,1.0,0.0,NaN,@gmail.com
4,4,1.0,1.0,2.0,33.0,@gmail.com


In [ ]:
user['age']=user['age'].fillna(user['age'].mean()) #meanwhile, for the NaN value inside the age column, I replaced with the mean of that column
# user['age']=user['age'].fillna(-1)

In [ ]:
user.info() # as we can see that there are no missing values remaining. But, look at the type of age which is supposed to integer instead of float.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127886 entries, 0 to 127885
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  127886 non-null  int64  
 1   attr_1   127886 non-null  float64
 2   attr_2   127886 non-null  float64
 3   attr_3   127886 non-null  float64
 4   age      127886 non-null  float64
 5   domain   127886 non-null  object 
dtypes: float64(4), int64(1), object(1)
memory usage: 5.9+ MB


In [ ]:
user['age']=user['age'].astype(int) # change the type of age into integer.

In [ ]:
user.age.value_counts()

31     52049
25      4064
24      3968
26      3962
27      3942
       ...  
104        1
99         1
97         1
91         1
108        1
Name: age, Length: 104, dtype: int64

In [ ]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127886 entries, 0 to 127885
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  127886 non-null  int64  
 1   attr_1   127886 non-null  float64
 2   attr_2   127886 non-null  float64
 3   attr_3   127886 non-null  float64
 4   age      127886 non-null  int64  
 5   domain   127886 non-null  object 
dtypes: float64(3), int64(2), object(1)
memory usage: 5.9+ MB


In [ ]:
user.shape

(127886, 6)

In [ ]:
train.head() # now we head back to the main dataset that is train. Let's apply some method to clean it.

,country_code,grass_date,user_id,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,open_flag,row_id
0,4,2019-07-16 00:00:00+08:00,43,44,19,6,18,0,2,4,12,43,99,0,5,10,0,0
1,4,2019-07-16 00:00:00+08:00,102,44,9,4,8,2,9,17,18,48,90,1,1,4,1,1
2,6,2019-07-16 00:00:00+08:00,177,49,14,5,5,0,4,12,24,69,119,5,19,27,0,2
3,1,2019-07-16 00:00:00+08:00,184,49,49,9,53,0,0,1,9,23,69,1,3,6,0,3
4,6,2019-07-16 00:00:00+08:00,221,49,227,6,221,0,0,0,2,5,5,0,0,0,0,4


In [ ]:
train.grass_date.unique() # to see the unique value of grass_date

array(['2019-07-16 00:00:00+08:00', '2019-07-17 00:00:00+08:00',
       '2019-07-18 00:00:00+08:00', '2019-07-19 00:00:00+08:00',
       '2019-07-20 00:00:00+08:00', '2019-07-21 00:00:00+08:00',
       '2019-07-22 00:00:00+08:00', '2019-07-23 00:00:00+08:00',
       '2019-07-24 00:00:00+08:00', '2019-07-25 00:00:00+08:00',
       '2019-07-26 00:00:00+08:00', '2019-07-27 00:00:00+08:00',
       '2019-07-28 00:00:00+08:00', '2019-07-29 00:00:00+08:00',
       '2019-07-30 00:00:00+08:00', '2019-08-01 00:00:00+08:00',
       '2019-08-02 00:00:00+08:00', '2019-08-03 00:00:00+08:00',
       '2019-08-04 00:00:00+08:00', '2019-08-05 00:00:00+08:00',
       '2019-08-06 00:00:00+08:00', '2019-08-07 00:00:00+08:00',
       '2019-08-08 00:00:00+08:00', '2019-08-09 00:00:00+08:00',
       '2019-08-10 00:00:00+08:00', '2019-08-11 00:00:00+08:00',
       '2019-08-12 00:00:00+08:00', '2019-08-13 00:00:00+08:00',
       '2019-08-14 00:00:00+08:00', '2019-08-15 00:00:00+08:00',
       '2019-08-16 00:00:

On the previous calculation of unique values, we see that the value contains both date and the time. So, I want to extract only the date by applying a function below.

In [ ]:
def date(text):
  text=text.split(' ')[0]
  return text

train['grass_date']=train['grass_date'].apply(date)

In [ ]:
train.head()

,country_code,grass_date,user_id,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,open_flag,row_id
0,4,2019-07-16,43,44,19,6,18,0,2,4,12,43,99,0,5,10,0,0
1,4,2019-07-16,102,44,9,4,8,2,9,17,18,48,90,1,1,4,1,1
2,6,2019-07-16,177,49,14,5,5,0,4,12,24,69,119,5,19,27,0,2
3,1,2019-07-16,184,49,49,9,53,0,0,1,9,23,69,1,3,6,0,3
4,6,2019-07-16,221,49,227,6,221,0,0,0,2,5,5,0,0,0,0,4


In [ ]:
import datetime
train['grass_date']=pd.to_datetime(train['grass_date']) # change the type of grass_date into date-time type

Next up, I want to do feature engineering on the dataset. I want to extract the day, month, week, and day of week from the grass_date because I think we need that information later on. After that, I remove the grass date column. 

In [ ]:
def extract_day(text):
  day=text.day
  return day

def extract_month(text):
  month=text.month
  return month

def extract_week(text):
  week=text.week
  return week

def extract_dow(text):
  dow=text.dayofweek
  return dow

train['day']=train['grass_date'].apply(extract_day)
train['month']=train['grass_date'].apply(extract_month)
train['week']=train['grass_date'].apply(extract_week)
train['day_of_week']=train['grass_date'].apply(extract_dow)

In [ ]:
train=train[['row_id','country_code','user_id','day','month','day_of_week','week','subject_line_length','last_open_day','last_login_day','last_checkout_day',
             'open_count_last_10_days','open_count_last_30_days','open_count_last_60_days','login_count_last_10_days','login_count_last_30_days',
             'login_count_last_60_days','checkout_count_last_10_days','checkout_count_last_30_days','checkout_count_last_60_days','open_flag']]

Then, let's check another columns and extract the necessary data from it to help us achieve the better performance of our model to be built later. On the last_open_day, last_login_day, and last_checkout_day, they include the value of "never" which means they never operate their account. So, in order to help us later, we are gonna to replace it with value '-1'. Last, we will convert the type into int as how it should be.

In [ ]:
train.last_open_day.value_counts() # there are 7000s 'never open' values and we will replace it with -1.

Never open    7518
1             5051
2             4429
3             3353
4             2848
              ... 
416              1
573              1
628              1
450              1
490              1
Name: last_open_day, Length: 549, dtype: int64

In [ ]:
train[train['last_open_day']=="Never open"]=train[train['last_open_day']=="Never open"].replace("Never open",'-1')
train['last_open_day']=train['last_open_day'].astype(int) #change the type into integer

In [ ]:
train.last_open_day.value_counts()

-1      7518
 1      5051
 2      4429
 3      3353
 4      2848
        ... 
 587       1
 545       1
 331       1
 737       1
 703       1
Name: last_open_day, Length: 549, dtype: int64

In [ ]:
train[train['last_login_day']=="Never login"]=train[train['last_login_day']=="Never login"].replace("Never login",'-1')
train['last_login_day']=train['last_login_day'].astype(int)

In [ ]:
train[train['last_checkout_day']=="Never checkout"]=train[train['last_checkout_day']=="Never checkout"].replace("Never checkout",'-1')
train['last_checkout_day']=train['last_checkout_day'].astype(int)

In [ ]:
train.shape # check the dimension of train

(73539, 21)

In [ ]:
train.head()

,row_id,country_code,user_id,day,month,day_of_week,week,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,open_flag
0,0,4,43,16,7,1,29,44,19,6,18,0,2,4,12,43,99,0,5,10,0
1,1,4,102,16,7,1,29,44,9,4,8,2,9,17,18,48,90,1,1,4,1
2,2,6,177,16,7,1,29,49,14,5,5,0,4,12,24,69,119,5,19,27,0
3,3,1,184,16,7,1,29,49,49,9,53,0,0,1,9,23,69,1,3,6,0
4,4,6,221,16,7,1,29,49,227,6,221,0,0,0,2,5,5,0,0,0,0


From the last result, we have changed all the values in the dataset by numeric. Next, we will merge dataset between train and user to obtain more comprehensive result in the dataset related to the users.

In [ ]:
df=pd.merge(train,user,how='inner',on='user_id') #merge both data set with 'user_id' with the merge key and result like below
df.head()

,row_id,country_code,user_id,day,month,day_of_week,week,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,open_flag,attr_1,attr_2,attr_3,age,domain
0,0,4,43,16,7,1,29,44,19,6,18,0,2,4,12,43,99,0,5,10,0,1.0,1.0,2.0,47,@gmail.com
1,1,4,102,16,7,1,29,44,9,4,8,2,9,17,18,48,90,1,1,4,1,1.0,1.0,2.0,25,@hotmail.com
2,2,6,177,16,7,1,29,49,14,5,5,0,4,12,24,69,119,5,19,27,0,1.0,1.0,0.0,31,@yahoo.com
3,3,1,184,16,7,1,29,49,49,9,53,0,0,1,9,23,69,1,3,6,0,1.0,1.0,2.0,24,@yahoo.com
4,4,6,221,16,7,1,29,49,227,6,221,0,0,0,2,5,5,0,0,0,0,1.0,1.0,0.0,31,@hotmail.com


In [ ]:
df.info() # now let's check info of the newly-merged dataset to prevent any loss of information

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73537 entries, 0 to 73536
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   row_id                       73537 non-null  int64  
 1   country_code                 73537 non-null  int64  
 2   user_id                      73537 non-null  int64  
 3   day                          73537 non-null  int64  
 4   month                        73537 non-null  int64  
 5   day_of_week                  73537 non-null  int64  
 6   week                         73537 non-null  int64  
 7   subject_line_length          73537 non-null  int64  
 8   last_open_day                73537 non-null  int64  
 9   last_login_day               73537 non-null  int64  
 10  last_checkout_day            73537 non-null  int64  
 11  open_count_last_10_days      73537 non-null  int64  
 12  open_count_last_30_days      73537 non-null  int64  
 13  open_count_last_

In [ ]:
df.set_index('row_id',inplace=True) #set 'row_id' as the new index of this new dataset

In [ ]:
df.index.unique()

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            73529, 73530, 73531, 73532, 73533, 73534, 73535, 73536, 73537,
            73538],
           dtype='int64', name='row_id', length=73537)

Because domain is the only non-numeric columns, so we need to change it to numeric by using encoding, that is one-hot encoding. We will use get_dummies to change it.

In [ ]:
df.domain.value_counts()

@gmail.com         62668
@yahoo.com          5181
@hotmail.com        2929
other               1199
@icloud.com          439
@qq.com              338
@outlook.com         259
@ymail.com           194
@live.com            169
@163.com             108
@rocketmail.com       53
Name: domain, dtype: int64

In [ ]:
#df=df.drop('domain',axis=1)
df=pd.get_dummies(df,columns=['domain'])

In [ ]:
df.head()

,country_code,user_id,day,month,day_of_week,week,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,open_flag,attr_1,attr_2,attr_3,age,domain_@163.com,domain_@gmail.com,domain_@hotmail.com,domain_@icloud.com,domain_@live.com,domain_@outlook.com,domain_@qq.com,domain_@rocketmail.com,domain_@yahoo.com,domain_@ymail.com,domain_other
row_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,4,43,16,7,1,29,44,19,6,18,0,2,4,12,43,99,0,5,10,0,1.0,1.0,2.0,47,0,1,0,0,0,0,0,0,0,0,0
1,4,102,16,7,1,29,44,9,4,8,2,9,17,18,48,90,1,1,4,1,1.0,1.0,2.0,25,0,0,1,0,0,0,0,0,0,0,0
2,6,177,16,7,1,29,49,14,5,5,0,4,12,24,69,119,5,19,27,0,1.0,1.0,0.0,31,0,0,0,0,0,0,0,0,1,0,0
3,1,184,16,7,1,29,49,49,9,53,0,0,1,9,23,69,1,3,6,0,1.0,1.0,2.0,24,0,0,0,0,0,0,0,0,1,0,0
4,6,221,16,7,1,29,49,227,6,221,0,0,0,2,5,5,0,0,0,0,1.0,1.0,0.0,31,0,0,1,0,0,0,0,0,0,0,0


Change all types in the columns into integer. Then, drop some columns unnecessary for the modelling.

In [ ]:
df['country_code']=df['country_code'].astype(int)
df['user_id']=df['user_id'].astype(int)
df['day']=df['day'].astype(int)
df['month']=df['month'].astype(int)
df['week']=df['week'].astype(int)
df['subject_line_length']=df['subject_line_length'].astype(int)
df['last_open_day']=df['last_open_day'].astype(int)
df['last_login_day']=df['last_login_day'].astype(int)
df['last_checkout_day']=df['last_checkout_day'].astype(int)
df['open_count_last_10_days']=df['open_count_last_10_days'].astype(int)
df['open_count_last_30_days']=df['open_count_last_30_days'].astype(int)
df['open_count_last_60_days']=df['open_count_last_60_days'].astype(int)
df['login_count_last_10_days']=df['login_count_last_10_days'].astype(int)
df['login_count_last_30_days']=df['login_count_last_30_days'].astype(int)
df['login_count_last_60_days']=df['login_count_last_60_days'].astype(int)
df['checkout_count_last_10_days']=df['checkout_count_last_10_days'].astype(int)
df['checkout_count_last_30_days']=df['checkout_count_last_30_days'].astype(int)
df['checkout_count_last_60_days']=df['checkout_count_last_60_days'].astype(int)
df['open_flag']=df['open_flag'].astype(int)

So, after doing some analysis and trial-error actions, I decided to drop some columns, that are open_flag because it is the label, user_id will give bias to the dataset and can be replaced by the row_id, for date-related columns we only keep day of week to the next step. Then the selected columns are placed in dataframe x, while y works as the label.

In [ ]:
x=df.drop(['open_flag','user_id','week','day','month'],axis=1)
y=df['open_flag']

In [ ]:
cols=[y for y in x.columns]
cols

['country_code',
 'day_of_week',
 'subject_line_length',
 'last_open_day',
 'last_login_day',
 'last_checkout_day',
 'open_count_last_10_days',
 'open_count_last_30_days',
 'open_count_last_60_days',
 'login_count_last_10_days',
 'login_count_last_30_days',
 'login_count_last_60_days',
 'checkout_count_last_10_days',
 'checkout_count_last_30_days',
 'checkout_count_last_60_days',
 'attr_1',
 'attr_2',
 'attr_3',
 'age',
 'domain_@163.com',
 'domain_@gmail.com',
 'domain_@hotmail.com',
 'domain_@icloud.com',
 'domain_@live.com',
 'domain_@outlook.com',
 'domain_@qq.com',
 'domain_@rocketmail.com',
 'domain_@yahoo.com',
 'domain_@ymail.com',
 'domain_other']

In [ ]:
from sklearn.preprocessing import StandardScaler # do standardization to keep all the columns the same 'chance' to be considered in the model.
scale=StandardScaler()
scaling=scale.fit_transform(x)
df_train=pd.DataFrame(scaling,columns=cols)
df_train

,country_code,day_of_week,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,attr_1,attr_2,attr_3,age,domain_@163.com,domain_@gmail.com,domain_@hotmail.com,domain_@icloud.com,domain_@live.com,domain_@outlook.com,domain_@qq.com,domain_@rocketmail.com,domain_@yahoo.com,domain_@ymail.com,domain_other
0,0.875626,-0.796457,0.030623,-0.193430,-0.136770,-0.326605,-0.519854,-0.170786,-0.129000,0.269477,0.468006,0.696031,-0.315127,0.286743,0.327530,0.012231,0.052353,0.890793,1.697450,-0.038351,0.416459,-0.203673,-0.077496,-0.047994,-0.059452,-0.067953,-0.026856,-0.275308,-0.051431,-0.128744
1,0.875626,-0.796457,0.030623,-0.381181,-0.142574,-0.411472,0.620359,1.360860,1.516082,0.716421,0.600095,0.570717,0.031201,-0.233676,-0.101646,0.012231,0.052353,0.890793,-0.677520,-0.038351,-2.401199,4.909839,-0.077496,-0.047994,-0.059452,-0.067953,-0.026856,-0.275308,-0.051431,-0.128744
2,2.069487,-0.796457,0.476285,-0.287306,-0.139672,-0.436932,-0.519854,0.266828,0.883358,1.163366,1.154868,0.974507,1.416514,2.108211,1.543530,0.012231,0.052353,-1.041146,-0.029801,-0.038351,-2.401199,-0.203673,-0.077496,-0.047994,-0.059452,-0.067953,-0.026856,3.632298,-0.051431,-0.128744
3,-0.915166,-0.796457,0.476285,0.369823,-0.128064,-0.029570,-0.519854,-0.608399,-0.508635,0.046005,-0.060349,0.278317,0.031201,0.026533,0.041413,0.012231,0.052353,0.890793,-0.785473,-0.038351,-2.401199,-0.203673,-0.077496,-0.047994,-0.059452,-0.067953,-0.026856,3.632298,-0.051431,-0.128744
4,2.069487,-0.796457,0.476285,3.711793,-0.136770,1.396197,-0.519854,-0.608399,-0.635179,-0.475430,-0.535868,-0.612805,-0.315127,-0.363781,-0.387764,0.012231,0.052353,-1.041146,-0.029801,-0.038351,-2.401199,4.909839,-0.077496,-0.047994,-0.059452,-0.067953,-0.026856,-0.275308,-0.051431,-0.128744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73532,2.069487,-1.288513,-0.415039,-0.099555,-0.049709,1.888426,-0.519854,-0.389592,-0.508635,-0.624411,-0.667957,-0.682424,-0.315127,-0.363781,-0.387764,0.012231,0.052353,-1.041146,-0.029801,-0.038351,-2.401199,4.909839,-0.077496,-0.047994,-0.059452,-0.067953,-0.026856,-0.275308,-0.051431,-0.128744
73533,-0.318235,-1.288513,-0.504172,0.313498,-0.125162,-0.046544,-0.519854,-0.608399,-0.508635,-0.624411,-0.667957,-0.682424,-0.315127,-0.363781,-0.387764,0.012231,0.052353,0.890793,-0.137754,-0.038351,0.416459,-0.203673,-0.077496,-0.047994,-0.059452,-0.067953,-0.026856,-0.275308,-0.051431,-0.128744
73534,-0.318235,-1.288513,-1.038966,-0.568932,-0.157084,-0.487852,-0.519854,-0.608399,-0.635179,-0.624411,-0.667957,-0.682424,-0.315127,-0.363781,-0.387764,0.012231,0.052353,-1.041146,-0.029801,-0.038351,0.416459,-0.203673,-0.077496,-0.047994,-0.059452,-0.067953,-0.026856,-0.275308,-0.051431,-0.128744
73535,-0.318235,-1.288513,-0.504172,-0.456282,-0.055514,-0.487852,0.620359,0.266828,-0.129000,-0.624411,-0.667957,-0.682424,-0.315127,-0.363781,-0.387764,0.012231,0.052353,-0.075176,0.725871,-0.038351,0.416459,-0.203673,-0.077496,-0.047994,-0.059452,-0.067953,-0.026856,-0.275308,-0.051431,-0.128744


In [ ]:
#month,day,week=extract_date(df.grass_date[0])

In [ ]:
#print(month,day,week)

In [ ]:
y

row_id
0        0
1        1
2        0
3        0
4        0
        ..
73534    0
73535    0
73536    0
73537    1
73538    0
Name: open_flag, Length: 73537, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73537 entries, 0 to 73538
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   country_code                 73537 non-null  int64  
 1   user_id                      73537 non-null  int64  
 2   day                          73537 non-null  int64  
 3   month                        73537 non-null  int64  
 4   day_of_week                  73537 non-null  int64  
 5   week                         73537 non-null  int64  
 6   subject_line_length          73537 non-null  int64  
 7   last_open_day                73537 non-null  int64  
 8   last_login_day               73537 non-null  int64  
 9   last_checkout_day            73537 non-null  int64  
 10  open_count_last_10_days      73537 non-null  int64  
 11  open_count_last_30_days      73537 non-null  int64  
 12  open_count_last_60_days      73537 non-null  int64  
 13  login_count_last

Next is building a model for our dataset based on the training set. After several trials, we land on Gradient Boosting Classifier as the algorithm because it generates higher score compared to others.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

#clf=KNeighborsClassifier(n_neighbors=5,weights='distance')
#clf=SVC(kernel='linear')
#clf=RandomForestClassifier()
#clf=MLPClassifier(alpha=0.1,activation='logistic')
clf=GradientBoostingClassifier()
cv_result=cross_val_score(clf,df_train,y,cv=10)

In [ ]:
cv_result.mean()

0.8796800463779592

In [ ]:
clf.fit(df_train,y) #fit the dataset to model

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

We have finished our training dataset. Now, it's time to apply the same things to our test dataset, including merge both test and user dataframe and scaling.

In [ ]:
test=pd.read_csv('/content/drive/My Drive/Shopee/Marketing Analytics/test.csv')

In [ ]:
test.head()

,country_code,grass_date,user_id,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,row_id
0,6,2019-09-03 00:00:00+08:00,0,35,27,2,13,2,3,4,10,34,134,0,6,18,0
1,6,2019-09-03 00:00:00+08:00,130,35,7,5,383,1,1,1,5,5,5,0,0,0,1
2,5,2019-09-03 00:00:00+08:00,150,25,34,1,3,0,0,0,13,19,38,2,2,2,2
3,1,2019-09-03 00:00:00+08:00,181,36,63,5,5,0,0,0,43,110,173,2,5,5,3
4,5,2019-09-03 00:00:00+08:00,192,23,6,5,54,0,0,0,4,12,39,0,0,2,4


In [ ]:
test.grass_date.unique()

array(['2019-09-03 00:00:00+08:00', '2019-09-04 00:00:00+08:00',
       '2019-09-05 00:00:00+08:00', '2019-09-06 00:00:00+08:00',
       '2019-09-07 00:00:00+08:00', '2019-09-08 00:00:00+08:00',
       '2019-09-09 00:00:00+08:00', '2019-09-10 00:00:00+08:00',
       '2019-09-11 00:00:00+08:00', '2019-09-12 00:00:00+08:00',
       '2019-09-13 00:00:00+08:00', '2019-09-14 00:00:00+08:00',
       '2019-09-16 00:00:00+08:00', '2019-09-17 00:00:00+08:00',
       '2019-09-18 00:00:00+08:00', '2019-09-19 00:00:00+08:00',
       '2019-09-20 00:00:00+08:00', '2019-09-21 00:00:00+08:00',
       '2019-09-22 00:00:00+08:00', '2019-09-23 00:00:00+08:00',
       '2019-09-24 00:00:00+08:00', '2019-09-25 00:00:00+08:00',
       '2019-09-26 00:00:00+08:00', '2019-09-27 00:00:00+08:00',
       '2019-09-28 00:00:00+08:00', '2019-09-29 00:00:00+08:00'],
      dtype=object)

In [ ]:
test['grass_date']=test['grass_date'].apply(date)

In [ ]:
test.head()

,country_code,grass_date,user_id,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,row_id
0,6,2019-09-03,0,35,27,2,13,2,3,4,10,34,134,0,6,18,0
1,6,2019-09-03,130,35,7,5,383,1,1,1,5,5,5,0,0,0,1
2,5,2019-09-03,150,25,34,1,3,0,0,0,13,19,38,2,2,2,2
3,1,2019-09-03,181,36,63,5,5,0,0,0,43,110,173,2,5,5,3
4,5,2019-09-03,192,23,6,5,54,0,0,0,4,12,39,0,0,2,4


In [ ]:
test['grass_date']=pd.to_datetime(test['grass_date'])

In [ ]:
test['day']=test['grass_date'].apply(extract_day)
test['month']=test['grass_date'].apply(extract_month)
test['week']=test['grass_date'].apply(extract_week)

In [ ]:
test['day_of_week']=test['grass_date'].apply(extract_dow)

In [ ]:
test=test[['row_id','country_code','user_id','day','month','day_of_week','week','subject_line_length','last_open_day','last_login_day','last_checkout_day',
             'open_count_last_10_days','open_count_last_30_days','open_count_last_60_days','login_count_last_10_days','login_count_last_30_days',
             'login_count_last_60_days','checkout_count_last_10_days','checkout_count_last_30_days','checkout_count_last_60_days']]

In [ ]:
test[test['last_open_day']=="Never open"]=test[test['last_open_day']=="Never open"].replace("Never open",'-1')
test['last_open_day']=test['last_open_day'].astype(int)
test[test['last_login_day']=="Never login"]=test[test['last_login_day']=="Never login"].replace("Never login",'-1')
test['last_login_day']=test['last_login_day'].astype(int)
test[test['last_checkout_day']=="Never checkout"]=test[test['last_checkout_day']=="Never checkout"].replace("Never checkout",'-1')
test['last_checkout_day']=test['last_checkout_day'].astype(int)

In [ ]:
test=pd.merge(test,user,how='inner',on='user_id')
test.head()

,row_id,country_code,user_id,day,month,day_of_week,week,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,attr_1,attr_2,attr_3,age,domain
0,0,6,0,3,9,1,36,35,27,2,13,2,3,4,10,34,134,0,6,18,1.0,1.0,0.0,31,@gmail.com
1,1,6,130,3,9,1,36,35,7,5,383,1,1,1,5,5,5,0,0,0,1.0,1.0,0.0,31,@gmail.com
2,2,5,150,3,9,1,36,25,34,1,3,0,0,0,13,19,38,2,2,2,1.0,1.0,2.0,33,@gmail.com
3,3,1,181,3,9,1,36,36,63,5,5,0,0,0,43,110,173,2,5,5,1.0,1.0,2.0,22,@yahoo.com
4,4,5,192,3,9,1,36,23,6,5,54,0,0,0,4,12,39,0,0,2,1.0,1.0,0.0,31,@gmail.com


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55970 entries, 0 to 55969
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   row_id                       55970 non-null  int64  
 1   country_code                 55970 non-null  int64  
 2   user_id                      55970 non-null  int64  
 3   day                          55970 non-null  int64  
 4   month                        55970 non-null  int64  
 5   day_of_week                  55970 non-null  int64  
 6   week                         55970 non-null  int64  
 7   subject_line_length          55970 non-null  int64  
 8   last_open_day                55970 non-null  int64  
 9   last_login_day               55970 non-null  int64  
 10  last_checkout_day            55970 non-null  int64  
 11  open_count_last_10_days      55970 non-null  int64  
 12  open_count_last_30_days      55970 non-null  int64  
 13  open_count_last_

In [ ]:
test.set_index('row_id',inplace=True)

In [ ]:
test=pd.get_dummies(test,columns=['domain'])

In [ ]:
test['country_code']=test['country_code'].astype(int)
test['user_id']=test['user_id'].astype(int)
test['day']=test['day'].astype(int)
test['month']=test['month'].astype(int)
test['week']=test['week'].astype(int)
test['subject_line_length']=test['subject_line_length'].astype(int)
test['last_open_day']=test['last_open_day'].astype(int)
test['last_login_day']=test['last_login_day'].astype(int)
test['last_checkout_day']=test['last_checkout_day'].astype(int)
test['open_count_last_10_days']=test['open_count_last_10_days'].astype(int)
test['open_count_last_30_days']=test['open_count_last_30_days'].astype(int)
test['open_count_last_60_days']=test['open_count_last_60_days'].astype(int)
test['login_count_last_10_days']=test['login_count_last_10_days'].astype(int)
test['login_count_last_30_days']=test['login_count_last_30_days'].astype(int)
test['login_count_last_60_days']=test['login_count_last_60_days'].astype(int)
test['checkout_count_last_10_days']=test['checkout_count_last_10_days'].astype(int)
test['checkout_count_last_30_days']=test['checkout_count_last_30_days'].astype(int)
test['checkout_count_last_60_days']=test['checkout_count_last_60_days'].astype(int)

In [ ]:
test=test.drop(['user_id','day','month','week'],axis=1)

In [ ]:
x.columns

Index(['country_code', 'day_of_week', 'subject_line_length', 'last_open_day',
       'last_login_day', 'last_checkout_day', 'open_count_last_10_days',
       'open_count_last_30_days', 'open_count_last_60_days',
       'login_count_last_10_days', 'login_count_last_30_days',
       'login_count_last_60_days', 'checkout_count_last_10_days',
       'checkout_count_last_30_days', 'checkout_count_last_60_days', 'attr_1',
       'attr_2', 'attr_3', 'age', 'domain_@163.com', 'domain_@gmail.com',
       'domain_@hotmail.com', 'domain_@icloud.com', 'domain_@live.com',
       'domain_@outlook.com', 'domain_@qq.com', 'domain_@rocketmail.com',
       'domain_@yahoo.com', 'domain_@ymail.com', 'domain_other'],
      dtype='object')

In [ ]:
test.columns

Index(['country_code', 'day_of_week', 'subject_line_length', 'last_open_day',
       'last_login_day', 'last_checkout_day', 'open_count_last_10_days',
       'open_count_last_30_days', 'open_count_last_60_days',
       'login_count_last_10_days', 'login_count_last_30_days',
       'login_count_last_60_days', 'checkout_count_last_10_days',
       'checkout_count_last_30_days', 'checkout_count_last_60_days', 'attr_1',
       'attr_2', 'attr_3', 'age', 'domain_@163.com', 'domain_@gmail.com',
       'domain_@hotmail.com', 'domain_@icloud.com', 'domain_@live.com',
       'domain_@outlook.com', 'domain_@qq.com', 'domain_@rocketmail.com',
       'domain_@yahoo.com', 'domain_@ymail.com', 'domain_other'],
      dtype='object')

In [ ]:
scaling2=scale.fit_transform(test)
df_test=pd.DataFrame(scaling2,columns=cols)
df_test

,country_code,day_of_week,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,attr_1,attr_2,attr_3,age,domain_@163.com,domain_@gmail.com,domain_@hotmail.com,domain_@icloud.com,domain_@live.com,domain_@outlook.com,domain_@qq.com,domain_@rocketmail.com,domain_@yahoo.com,domain_@ymail.com,domain_other
0,1.830971,-0.749114,-0.587544,-0.021621,-0.165330,-0.361208,0.699230,0.116595,-0.146192,0.073062,0.199898,1.136414,-0.321274,0.339208,0.847721,0.008454,0.031076,-1.048146,-0.018621,-0.036385,0.434804,-0.197147,-0.080234,-0.049535,-0.053543,-0.087886,-0.033031,-0.291917,-0.052527,-0.137396
1,1.830971,-0.749114,-0.587544,-0.422127,-0.154448,2.815955,0.128769,-0.301217,-0.477559,-0.284701,-0.551936,-0.623752,-0.321274,-0.392734,-0.416894,0.008454,0.031076,-1.048146,-0.018621,-0.036385,0.434804,-0.197147,-0.080234,-0.049535,-0.053543,-0.087886,-0.033031,-0.291917,-0.052527,-0.137396
2,1.252426,-0.749114,-1.402140,0.118556,-0.168958,-0.447077,-0.441692,-0.510124,-0.588015,0.287721,-0.188982,-0.173477,0.250045,-0.148753,-0.276381,0.008454,0.031076,0.882536,0.205773,-0.036385,0.434804,-0.197147,-0.080234,-0.049535,-0.053543,-0.087886,-0.033031,-0.291917,-0.052527,-0.137396
3,-1.061755,-0.749114,-0.506084,0.699291,-0.154448,-0.429903,-0.441692,-0.510124,-0.588015,2.434301,2.170224,1.668557,0.250045,0.217218,-0.065612,0.008454,0.031076,0.882536,-1.028392,-0.036385,-2.299885,-0.197147,-0.080234,-0.049535,-0.053543,-0.087886,-0.033031,3.425628,-0.052527,-0.137396
4,1.252426,-0.749114,-1.565059,-0.442152,-0.154448,-0.009144,-0.441692,-0.510124,-0.588015,-0.356254,-0.370459,-0.159832,-0.321274,-0.392734,-0.276381,0.008454,0.031076,-1.048146,-0.018621,-0.036385,0.434804,-0.197147,-0.080234,-0.049535,-0.053543,-0.087886,-0.033031,-0.291917,-0.052527,-0.137396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55965,1.830971,1.793728,0.878729,-0.582329,-0.158075,-0.404143,-0.441692,-0.510124,-0.588015,-0.642464,-0.681563,-0.691976,-0.321274,-0.392734,-0.416894,0.008454,0.031076,-1.048146,-0.018621,-0.036385,0.434804,-0.197147,-0.080234,-0.049535,-0.053543,-0.087886,-0.033031,-0.291917,-0.052527,-0.137396
55966,1.830971,1.793728,0.878729,0.619189,2.736698,9.891583,-0.441692,-0.510124,-0.477559,-0.642464,-0.681563,-0.691976,-0.321274,-0.392734,-0.416894,0.008454,0.031076,-1.048146,-0.018621,-0.036385,0.434804,-0.197147,-0.080234,-0.049535,-0.053543,-0.087886,-0.033031,-0.291917,-0.052527,-0.137396
55967,1.830971,1.793728,0.064133,-0.582329,-0.147193,-0.481425,-0.441692,-0.510124,-0.588015,-0.642464,-0.681563,-0.691976,-0.321274,-0.392734,-0.416894,0.008454,0.031076,-1.048146,-0.018621,-0.036385,-2.299885,5.072351,-0.080234,-0.049535,-0.053543,-0.087886,-0.033031,-0.291917,-0.052527,-0.137396
55968,1.830971,1.793728,0.064133,-0.462178,-0.154448,-0.421316,0.699230,0.534407,0.958367,-0.642464,-0.681563,-0.691976,-0.321274,-0.392734,-0.416894,0.008454,0.031076,-1.048146,-0.018621,-0.036385,0.434804,-0.197147,-0.080234,-0.049535,-0.053543,-0.087886,-0.033031,-0.291917,-0.052527,-0.137396


In [ ]:
test['open_flag']=clf.predict(df_test) # last one step is predict the test dataset using our built model.

In [ ]:
test.head()

,country_code,day_of_week,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,attr_1,attr_2,attr_3,age,domain_@163.com,domain_@gmail.com,domain_@hotmail.com,domain_@icloud.com,domain_@live.com,domain_@outlook.com,domain_@qq.com,domain_@rocketmail.com,domain_@yahoo.com,domain_@ymail.com,domain_other,open_flag
row_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,6,1,35,27,2,13,2,3,4,10,34,134,0,6,18,1.0,1.0,0.0,31,0,1,0,0,0,0,0,0,0,0,0,0
1,6,1,35,7,5,383,1,1,1,5,5,5,0,0,0,1.0,1.0,0.0,31,0,1,0,0,0,0,0,0,0,0,0,0
2,5,1,25,34,1,3,0,0,0,13,19,38,2,2,2,1.0,1.0,2.0,33,0,1,0,0,0,0,0,0,0,0,0,0
3,1,1,36,63,5,5,0,0,0,43,110,173,2,5,5,1.0,1.0,2.0,22,0,0,0,0,0,0,0,0,1,0,0,0
4,5,1,23,6,5,54,0,0,0,4,12,39,0,0,2,1.0,1.0,0.0,31,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
test['row_id']=test.index
test.head()

,country_code,day_of_week,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,attr_1,attr_2,attr_3,age,domain_@163.com,domain_@gmail.com,domain_@hotmail.com,domain_@icloud.com,domain_@live.com,domain_@outlook.com,domain_@qq.com,domain_@rocketmail.com,domain_@yahoo.com,domain_@ymail.com,domain_other,open_flag,row_id
row_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,6,1,35,27,2,13,2,3,4,10,34,134,0,6,18,1.0,1.0,0.0,31,0,1,0,0,0,0,0,0,0,0,0,0,0
1,6,1,35,7,5,383,1,1,1,5,5,5,0,0,0,1.0,1.0,0.0,31,0,1,0,0,0,0,0,0,0,0,0,0,1
2,5,1,25,34,1,3,0,0,0,13,19,38,2,2,2,1.0,1.0,2.0,33,0,1,0,0,0,0,0,0,0,0,0,0,2
3,1,1,36,63,5,5,0,0,0,43,110,173,2,5,5,1.0,1.0,2.0,22,0,0,0,0,0,0,0,0,1,0,0,0,3
4,5,1,23,6,5,54,0,0,0,4,12,39,0,0,2,1.0,1.0,0.0,31,0,1,0,0,0,0,0,0,0,0,0,0,4


In [ ]:
submit=test[['row_id','open_flag']]
submit

,row_id,open_flag
row_id,,
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
55965,55965,0
55966,55966,0
55967,55967,0


In [ ]:
submit.to_csv('submissions3.csv',index=False)